## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [6]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [7]:
# Always remember to do this!
load_dotenv(override=True)

True

endpoint = "https://models.github.ai/inference"
model = "deepseek/DeepSeek-V3-0324"
token = os.environ["GITHUB_TOKEN"]

endpoint = "https://models.github.ai/inference"
model = "meta/Llama-4-Scout-17B-16E-Instruct"
token = os.environ["GITHUB_TOKEN"]

endpoint = "https://models.github.ai/inference"
model = "openai/gpt-5"
token = os.environ["GITHUB_TOKEN"]

In [9]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-or-v
Google API Key exists and begins AI
DeepSeek API Key exists and begins ghp
Groq API Key exists and begins gith


In [10]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [11]:
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [12]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


How would you approach the task of reconciling conflicting cultural values when addressing a global ethical dilemma, and what frameworks would you utilize to ensure a balanced perspective?


In [13]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [9]:
# The API we know well

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing a system that balances the ethical implications of AI decision-making with the need for efficiency and objectivity in critical areas like healthcare and criminal justice requires a multifaceted approach. Here’s a framework for establishing such a system:

### 1. **Establish Ethical Principles**
   - **Transparency**: Ensure that AI algorithms are transparent and that stakeholders can understand how decisions are made.
   - **Accountability**: Define clear lines of accountability for AI decisions. Who is responsible when an AI system makes a mistake?
   - **Fairness**: Minimize bias in the algorithm's outputs. Use diverse datasets to train AI and regularly test for fairness across different demographics.
   - **Beneficence**: Prioritize the welfare of individuals affected by AI decisions, particularly vulnerable populations in healthcare and criminal justice.
   - **Informed Consent**: In healthcare settings, ensure that patients are informed about the use of AI tools and give consent for their implementation in decision-making processes.

### 2. **Stakeholder Engagement**
   - **Diverse Perspectives**: Involve a wide range of stakeholders, including ethicists, community representatives, legal experts, healthcare professionals, and criminal justice advocates, in the design and deployment of AI systems.
   - **Public Consultation**: Conduct public engagement activities to gather input and understand societal values and concerns regarding AI use in sensitive contexts.

### 3. **Interdisciplinary Approach**
   - **Collaboration**: Create collaborative structures between developers, ethicists, domain experts (e.g., healthcare providers or legal professionals), and sociologists to ensure a comprehensive understanding of the implications of AI.
   - **Continuous Learning**: Implement ongoing research initiatives to assess the impact of AI decisions and ensure that the system adapts to new findings or societal changes.

### 4. **Robust Data Management**
   - **Quality of Data**: Ensure that the data used for training AI systems is high-quality, representative, and inclusive, avoiding historical biases that could perpetuate discrimination.
   - **Data Governance**: Establish strict data governance policies that protect privacy and ensure ethical data use, especially when dealing with sensitive information.

### 5. **Algorithmic Design and Testing**
   - **Bayesian Approaches**: Use probabilistic models that allow for uncertainty and can adapt over time as more data is collected.
   - **Simulation and Testing**: Rigorously test AI systems in controlled environments to evaluate potential biases, unintended consequences, and their generalizability before real-world deployment.

### 6. **Iterative Feedback Mechanism**
   - **Real-Time Monitoring**: Implement monitoring systems to track AI performance and outcomes continuously, allowing for real-time adjustments and error correction.
   - **Feedback Loops**: Create mechanisms for users to provide feedback about the AI’s recommendations to improve the system.

### 7. **Legal and Regulatory Framework**
   - **Compliance**: Ensure that AI systems comply with existing laws and regulations in healthcare and criminal justice while advocating for updates in legislation to address gaps around AI use.
   - **Ethics Review Committees**: Establish independent ethics boards that review AI systems and their deployment in critical sectors.

### 8. **Education and Training**
   - **Training Programs**: Develop educational programs for professionals in healthcare and criminal justice focused on understanding AI technologies, their strengths, limitations, and ethical considerations.
   - **Empowerment**: Equip practitioners with the tools and understanding necessary to interpret AI outputs and make informed decisions.

### 9. **Public Awareness Campaigns**
   - **Information Dissemination**: Create initiatives to raise public awareness about the role of AI in these sectors, fostering an informed discussion about its benefits and risks.
   - **Trust Building**: Work to build public trust through transparent practices, accountability, and clearly communicated values.

### Conclusion
This framework outlines a proactive and structured approach to designing an AI decision-making system that balances ethical considerations with efficiency. By fostering transparency, accountability, stakeholder engagement, and continuous evaluations, we can work towards fostering trust and ensuring that AI serves to enhance, rather than undermine, the moral imperatives of the healthcare and criminal justice systems.

In [14]:
# The API we know well

# model_name = "gpt-4o-mini"

# response = openai.chat.completions.create(model=model_name, messages=messages)
# answer = response.choices[0].message.content


# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)



claude = OpenAI(api_key=anthropic_api_key, base_url="https://openrouter.ai/api/v1")
model_name="anthropic/claude-sonnet-4"

response = claude.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answer.append(answer)

I'd approach this complex challenge through several complementary frameworks:

## Multi-layered Analysis Framework

**1. Identify Core vs. Surface Values**
- Distinguish between fundamental human needs (dignity, safety, autonomy) and their cultural expressions
- Look for shared underlying concerns that manifest differently across cultures
- Map where conflicts are truly irreconcilable vs. where they reflect different paths to similar goals

**2. Stakeholder Inclusion Process**
- Engage representatives from affected communities early and meaningfully
- Create space for cultures to articulate their own values rather than having them interpreted by others
- Use culturally appropriate dialogue methods

## Philosophical Frameworks

**Principlist Approach**: Balance competing principles (autonomy, beneficence, justice, non-maleficence) while recognizing their cultural interpretations vary

**Capabilities Approach**: Focus on what enables human flourishing across cultures - education, health, political participation - while allowing diverse paths to achieve these

**Contextual Ethics**: Acknowledge that universal principles may require culturally sensitive applications

## Practical Methods

- **Overlapping Consensus**: Seek solutions different cultures can support for their own reasons
- **Procedural Justice**: When substantive agreement is impossible, focus on fair, inclusive decision-making processes
- **Graduated Implementation**: Allow for regional variations within broader ethical frameworks

The goal isn't forcing uniformity but finding workable approaches that respect core human dignity while honoring meaningful cultural differences. What specific dilemma did you have in mind?

AttributeError: 'str' object has no attribute 'append'

In [13]:
# gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
# model_name = "gemini-2.0-flash"

# response = gemini.chat.completions.create(model=model_name, messages=messages)
# answer = response.choices[0].message.content

# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)

gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Balancing the ethical implications of AI decision-making with efficiency and objectivity in critical areas like healthcare and criminal justice requires a multi-faceted approach, incorporating both technological and societal safeguards.  Here's a design outlining the key components:

**I. Core Principles and Framework:**

*   **Transparency and Explainability:**  AI models should be understandable, allowing users to comprehend the reasoning behind decisions. Prioritize explainable AI (XAI) techniques.
*   **Fairness and Non-discrimination:**  Actively combat bias in data, algorithms, and outcomes.  Implement fairness metrics and audit for disparate impact on protected groups.
*   **Accountability and Auditability:**  Establish clear lines of responsibility for AI system design, deployment, and outcomes.  Maintain detailed logs of decisions and data usage for auditing purposes.
*   **Human Oversight and Control:**  Maintain a 'human-in-the-loop' or 'human-on-the-loop' approach for critical decisions. AI should augment, not replace, human judgment, particularly where significant consequences are involved.  Users should have the ability to override AI decisions based on ethical or contextual considerations.
*   **Data Privacy and Security:**  Protect sensitive data used in AI systems, adhering to privacy regulations like GDPR and HIPAA. Employ robust security measures to prevent unauthorized access and misuse.
*   **Beneficence and Non-maleficence:**  Ensure AI systems are designed to benefit individuals and society while minimizing potential harm.  Regularly assess and mitigate risks associated with AI deployment.
*   **Continuous Monitoring and Evaluation:**  Implement ongoing monitoring of AI system performance, fairness, and ethical implications. Adapt and improve the system based on feedback and evolving societal norms.

**II. System Components:**

1.  **Data Management and Bias Mitigation:**

    *   **Diverse and Representative Datasets:** Actively seek and curate datasets that accurately reflect the population affected by the AI system.
    *   **Bias Detection and Mitigation Tools:** Employ tools to identify and mitigate bias in data, algorithms, and outcomes. This includes techniques like re-weighting data, adversarial debiasing, and fairness-aware machine learning.
    *   **Data Auditing and Validation:**  Establish rigorous data auditing processes to ensure data quality, accuracy, and completeness. Regularly validate data sources to identify and correct errors or inconsistencies.
    *   **Data Anonymization and De-identification:** Implement robust anonymization techniques to protect sensitive patient or individual data while still enabling meaningful analysis.

2.  **AI Model Development and Deployment:**

    *   **XAI Techniques:** Prioritize the use of explainable AI (XAI) techniques to provide transparency into the model's decision-making process. Examples include LIME, SHAP, and attention mechanisms.
    *   **Model Validation and Testing:** Conduct rigorous testing and validation of AI models using diverse datasets and scenarios to assess their accuracy, robustness, and fairness.
    *   **Fairness Metrics:** Incorporate fairness metrics into the model evaluation process to identify and address potential biases. Examples include demographic parity, equal opportunity, and predictive parity.
    *   **Modular and Auditable Architecture:** Design the AI system with a modular architecture that allows for easy auditing and modification of individual components.
    *   **Version Control and Model Governance:** Implement a robust version control system to track changes to the AI model and data, ensuring accountability and reproducibility.
    *   **'Human-in-the-Loop' Integration:**  Develop interfaces that seamlessly integrate AI-generated recommendations with human decision-making processes.
    *   **Alerting and Monitoring Systems:**  Implement real-time monitoring systems that detect anomalies, biases, or unexpected behavior in the AI system.

3.  **Human Oversight and Governance:**

    *   **Ethics Review Boards:** Establish ethics review boards comprised of experts in AI, ethics, law, and relevant domain knowledge (e.g., healthcare, criminal justice). These boards would review AI system designs, deployment plans, and ongoing performance.
    *   **Clear Roles and Responsibilities:** Define clear roles and responsibilities for individuals involved in the AI system lifecycle, including developers, data scientists, ethicists, domain experts, and end-users.
    *   **Training and Education:** Provide comprehensive training to all stakeholders on the ethical implications of AI, data privacy, and responsible AI practices.
    *   **Feedback Mechanisms:** Establish mechanisms for individuals affected by AI decisions to provide feedback and challenge outcomes.
    *   **Appeal Processes:**  Develop clear and accessible appeal processes for individuals who believe they have been unfairly treated by an AI system.
    *   **Regulatory Frameworks:** Advocate for the development of clear and consistent regulatory frameworks for AI deployment in critical areas, addressing issues such as bias, transparency, accountability, and data privacy.

4.  **Technology Infrastructure:**

    *   **Secure Data Storage and Processing:** Implement secure data storage and processing infrastructure to protect sensitive data from unauthorized access and breaches.
    *   **Auditing and Logging Capabilities:**  Develop robust auditing and logging capabilities to track all AI system activity, including data access, model updates, and decision-making processes.
    *   **API Integration:** Design APIs that allow for seamless integration with existing healthcare or criminal justice systems.
    *   **Scalability and Reliability:**  Ensure that the AI system is scalable and reliable to meet the demands of real-world deployment.

**III.  Specific Considerations for Healthcare and Criminal Justice:**

*   **Healthcare:**
    *   Prioritize patient autonomy and informed consent.  Explain AI-driven diagnoses and treatment recommendations clearly to patients.
    *   Address concerns about job displacement among healthcare professionals.  Focus on AI as a tool to augment, not replace, human expertise.
    *   Ensure AI systems are used to improve access to healthcare for underserved populations.
*   **Criminal Justice:**
    *   Be extremely cautious about using AI for predictive policing or risk assessment, given the potential for perpetuating existing biases in the criminal justice system.
    *   Focus on using AI to improve the accuracy and fairness of forensic analysis and evidence processing.
    *   Ensure transparency and accountability in the use of AI for sentencing recommendations and parole decisions.

**IV. Ongoing Monitoring and Improvement:**

*   **Regular Audits:** Conduct regular audits of AI system performance, fairness, and ethical implications.
*   **Feedback Loops:**  Establish feedback loops to continuously improve the AI system based on user feedback and real-world experience.
*   **Adaptive Learning:**  Implement adaptive learning techniques to allow the AI system to learn from its mistakes and improve its performance over time.
*   **Staying Current with Research:**  Stay abreast of the latest research and developments in AI ethics, fairness, and transparency, and incorporate these advances into the AI system.

**Challenges:**

*   **Defining and Measuring Fairness:** Fairness is a complex and multifaceted concept, and there is no single definition that is universally accepted.
*   **Balancing Accuracy and Explainability:** Explainable AI techniques can sometimes sacrifice accuracy.
*   **Maintaining Data Privacy:** Striking the right balance between data privacy and the need for data to train and improve AI models is a challenge.
*   **Addressing Unforeseen Consequences:** AI systems can sometimes produce unexpected and unintended consequences.
*   **Evolving Societal Norms:** Ethical standards and societal norms are constantly evolving, which requires ongoing monitoring and adaptation of AI systems.

By implementing these components and continually evaluating the system's impact, we can strive to harness the power of AI for good while mitigating its potential risks and ensuring that it serves the best interests of individuals and society. This requires a collaborative effort involving AI developers, ethicists, policymakers, and the communities most affected by AI decisions.


In [17]:
# deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
# model_name = "deepseek-chat"

# response = deepseek.chat.completions.create(model=model_name, messages=messages)
# answer = response.choices[0].message.content

# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)

# endpoint = "https://models.github.ai/inference"
# model = "deepseek/DeepSeek-V3-0324"

deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://models.github.ai/inference")
model_name = "deepseek/DeepSeek-V3-0324"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing a system to balance ethical AI decision-making with efficiency and objectivity in critical domains like healthcare and criminal justice requires a multidisciplinary approach. Below is a structured framework to achieve this balance:

### **1. Foundational Ethical Principles**
- **Transparency**: Ensure AI decision-making processes are explainable (e.g., using interpretable models like decision trees or SHAP/LIME for black-box models).  
- **Fairness**: Mitigate biases by auditing datasets, using fairness-aware algorithms, and ensuring diverse representation in training data.  
- **Accountability**: Assign clear responsibility for AI decisions (human-in-the-loop oversight).  
- **Privacy & Consent**: Comply with regulations (e.g., GDPR, HIPAA) and ensure data anonymization where applicable.  
- **Non-Maleficence**: Implement safeguards to prevent harm (e.g., fail-safes in medical diagnosis AI).  

### **2. System Design Components**
#### **A. Data & Model Development**
- **Bias Detection & Mitigation**  
  - Use fairness metrics (e.g., demographic parity, equalized odds).  
  - Employ adversarial debiasing or reweighting techniques.  
- **High-Quality, Representative Data**  
  - Ensure datasets reflect diverse populations (e.g., racial, socioeconomic).  
  - Continuously audit for drift or emerging biases.  

#### **B. Decision-Making Process**
- **Human-AI Collaboration**  
  - **Healthcare**: AI for diagnostics (e.g., radiology) but require clinician validation.  
  - **Criminal Justice**: Use risk assessment tools (e.g., COMPAS) with judicial oversight.  
- **Explainability**  
  - Provide clear reasoning (e.g., "Patient flagged due to X symptoms matching Y condition with 85% confidence").  
  - Avoid opaque "black boxes" in high-stakes decisions.  

#### **C. Governance & Compliance**
- **Ethics Review Boards**  
  - Include ethicists, domain experts, and community representatives.  
  - Conduct impact assessments before deployment.  
- **Regulatory Alignment**  
  - Follow sector-specific guidelines (e.g., FDA for healthcare AI, ACLU recommendations for criminal justice).  
- **Audit Trails**  
  - Log all AI decisions for retrospective review.  

### **3. Continuous Monitoring & Feedback**
- **Real-World Performance Tracking**  
  - Monitor for disparities (e.g., higher false positives in certain demographics).  
- **Feedback Loops**  
  - Allow end-users (doctors, judges) to contest or correct AI outputs.  
- **Iterative Improvement**  
  - Update models based on new data and ethical insights.  

### **4. Public Trust & Stakeholder Engagement**
- **Community Involvement**  
  - Engage affected groups (e.g., patients, marginalized communities) in design.  
- **Transparency Reports**  
  - Publish performance metrics and bias audits.  

### **Example Implementations**
- **Healthcare**: IBM Watson for Oncology (with physician validation).  
- **Criminal Justice**: Risk assessment tools with judicial discretion (avoiding sole reliance on AI).  

### **Challenges & Mitigations**
- **Efficiency vs. Ethics Trade-offs**: Optimize for "sufficient" explainability (e.g., simpler models where possible).  
- **Adversarial Attacks**: Secure models against manipulation (e.g., in forensic AI).  

### **Conclusion**
A balanced AI system for critical domains must prioritize ethical safeguards without sacrificing utility. By combining **technical rigor (fairness-aware algorithms), human oversight, and robust governance**, we can achieve **objective yet morally sound outcomes**.  

Would you like a deeper dive into any specific component (e.g., bias mitigation techniques or regulatory frameworks)?

In [18]:
# groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
# model_name = "llama-3.3-70b-versatile"

# response = groq.chat.completions.create(model=model_name, messages=messages)
# answer = response.choices[0].message.content

# display(Markdown(answer))
# competitors.append(model_name)
# answers.append(answer)

# endpoint = "https://models.github.ai/inference"
# model = "meta/Llama-4-Scout-17B-16E-Instruct"

groq = OpenAI(api_key=groq_api_key, base_url="https://models.github.ai/inference")
model_name = "meta/Llama-4-Scout-17B-16E-Instruct"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

Designing a system to balance the ethical implications of AI decision-making with the need for efficient and objective outcomes in critical areas such as healthcare and criminal justice requires a multi-faceted approach. Here's a comprehensive framework to achieve this balance:

**Principles**

1. **Transparency**: Ensure that AI decision-making processes are transparent, explainable, and auditable.
2. **Accountability**: Establish clear accountability mechanisms for AI-driven decisions.
3. **Fairness**: Implement fairness and bias-detection mechanisms to prevent discriminatory outcomes.
4. **Human Oversight**: Involve human experts in the decision-making process to review and correct AI-driven decisions.
5. **Continuous Evaluation**: Regularly assess and update AI systems to ensure they remain aligned with societal values and norms.

**System Components**

1. **Data Curation**: Ensure that datasets used for AI training are diverse, representative, and free from bias.
2. **Algorithmic Auditing**: Regularly audit AI algorithms for bias, fairness, and transparency.
3. **Explainability Mechanisms**: Implement techniques such as model interpretability, feature attribution, and model-agnostic explanations to provide insights into AI-driven decisions.
4. **Human-in-the-Loop**: Involve human experts in the decision-making process to review and correct AI-driven decisions.
5. **Feedback Mechanisms**: Establish feedback loops to collect data on AI-driven decisions and their outcomes, enabling continuous evaluation and improvement.

**Healthcare-Specific Considerations**

1. **Clinical Validation**: Validate AI-driven decisions against established clinical guidelines and outcomes.
2. **Patient-Centered Care**: Ensure that AI systems prioritize patient-centered care and respect patient autonomy.
3. **Medical Liability**: Establish clear guidelines for medical liability in AI-driven decision-making.

**Criminal Justice-Specific Considerations**

1. **Due Process**: Ensure that AI-driven decisions in criminal justice adhere to due process principles.
2. **Bias Detection**: Implement bias-detection mechanisms to prevent discriminatory outcomes in AI-driven decision-making.
3. **Transparency in Sentencing**: Provide transparent explanations for AI-driven sentencing recommendations.

**Implementation Roadmap**

1. **Establish an Ethics Committee**: Form a committee to oversee AI development and deployment, comprising experts from diverse fields.
2. **Develop and Deploy AI Systems**: Develop and deploy AI systems that incorporate the principles and components outlined above.
3. **Monitor and Evaluate**: Continuously monitor and evaluate AI systems, making adjustments as needed to ensure they remain aligned with societal values and norms.
4. **Foster Collaboration**: Encourage collaboration between AI developers, domain experts, and stakeholders to ensure that AI systems meet the needs of critical areas like healthcare and criminal justice.

**Challenges and Limitations**

1. **Complexity**: AI systems can be complex and difficult to interpret, making it challenging to ensure transparency and accountability.
2. **Data Quality**: AI systems are only as good as the data they're trained on, which can be a challenge in areas with limited or biased data.
3. **Regulatory Frameworks**: Existing regulatory frameworks may not be equipped to handle the unique challenges posed by AI decision-making.

By following this framework, we can design AI systems that balance the need for efficient and objective outcomes with the need to address ethical implications in critical areas like healthcare and criminal justice.

## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [19]:
!ollama pull llama3.2

'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

NameError: name 'OpenAI' is not defined

In [ ]:
# So where are we?

print(competitors)
print(answers)


In [ ]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


In [20]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [22]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

In [29]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>